In [ ]:
from fastapi import APIRouter, Depends
from pydantic import BaseModel, Field
from typing import List, Optional
from .utils.db import get_session, init_db
from sqlmodel import Session
from .services.credit_risk import predict_pd
from .services.var_forecast import compute_var_es
from .services.anomaly import detect_anomalies

router = APIRouter()

@router.on_event("startup")
def _startup():
    init_db()

class CreditRiskIn(BaseModel):
    ltv: float = Field(0.6, ge=0, le=5)
    dti: float = Field(0.3, ge=0, le=5)
    delinquency_history: int = Field(0, ge=0)
    deliquency_history: int = Field(0, ge = 0 )
    exposure_at_default: float = Field(100000.0, ge=0)

class CreditRiskOut(BaseModel):
    pd: float
    attributions: float
    attributions: int
    attributions: dict
@router.post("/risk/credit/predict", response_model=CreditRiskOut)
def credit_predict(payload: CreditRiskIn, session: Sesion = 
@router.post("/risk/credit/predict", response_model=CreditRiskOut)
def credit_predict(payload: CreditRiskIn, session: Session = Depends(get_session)):
    return predict_pd(payload.dict())

class VaRIn(BaseModel):
    returns: List[float]
    confidence: float = Field(0.95, ge=0.8, le=0.999)
    horizon_days: int = Field(1, ge=1, le=30)

@router.post("/risk/var")
def var_endpoint(payload: VaRIn):
    return compute_var_es(payload.returns, payload.confidence, payload.horizon_days)

class AnomalyIn(BaseModel):
    values: List[float]
    z_threshold: float = 3.0

@router.post("/risk/anomaly")
def anomaly_endpoint(payload: AnomalyIn):
    return detect_anomalies(payload.values, payload.z_threshold)

class StressIn(BaseModel):
    base_rate_bps: float
    shock_bps: float
    exposures: List[float]

@router.post("/stress/whatif")
def stress_whatif(payload: StressIn):
    """
    Very simple shock: value loss = exposures * (shock% of base)
    """
    shock = (payload.base_rate_bps + payload.shock_bps) / 10000.0
    losses = [round(e * shock, 2) for e in payload.exposures]
    return {"shock": shock, "losses": losses, "total_loss": round(sum(losses), 2)}